In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn import preprocessing

import pandas as pd # for data manipulation
import numpy as np # for data manipulation

from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn import tree # for decision tree models

import plotly.express as px  # for data visualization
import plotly.graph_objects as go # for data visualization
import graphviz # for plotting decision tree graphs

# clean and droping of useless rows(for our model) 

In [4]:
df = pd.read_csv('drive/My Drive/statistics/Fraud.csv')

In [12]:
df[(df["Dest_diff"]==0) & (df["newbalanceDest"]!=0) &(df["oldbalanceDest"]!=0) ]

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,Dest_diff
3760289,279,CASH_OUT,0.0,C539112012,0.0,0.0,C1106468520,538547.63,538547.63,1,0,0.0
5563714,387,CASH_OUT,0.0,C1294472700,0.0,0.0,C1325541393,7970766.57,7970766.57,1,0,0.0
5996408,425,CASH_OUT,0.0,C832555372,0.0,0.0,C1462759334,76759.90,76759.90,1,0,0.0
5996410,425,CASH_OUT,0.0,C69493310,0.0,0.0,C719711728,2921531.34,2921531.34,1,0,0.0
6168500,554,CASH_OUT,0.0,C10965156,0.0,0.0,C1493336195,230289.66,230289.66,1,0,0.0
6205440,586,CASH_OUT,0.0,C1303719003,0.0,0.0,C900608348,1328472.86,1328472.86,1,0,0.0
6296015,671,CASH_OUT,0.0,C1960007029,0.0,0.0,C459118517,27938.72,27938.72,1,0,0.0
6351226,702,CASH_OUT,0.0,C1461113533,0.0,0.0,C1382150537,107777.02,107777.02,1,0,0.0
6362461,730,CASH_OUT,0.0,C729003789,0.0,0.0,C1388096959,1008609.53,1008609.53,1,0,0.0
6362585,741,CASH_OUT,0.0,C312737633,0.0,0.0,C1400061387,267522.87,267522.87,1,0,0.0


if type is payment or debit o type so the operation isn't fraud

In [15]:
df=df.drop(df[(df["type"]=="PAYMENT") | (df["type"]=="DEBIT") | (df["type"]=="CASH_IN")].index) #aren't Fraud

In [16]:
#new usefull feature
df['Dest_diff']=df['newbalanceDest']-df['oldbalanceDest'] 

if the old and new balance isn't null and dest difference is zero so it's a fraud




In [17]:
#is Fraud
df=df.drop(df[(df["Dest_diff"]==0) & (df["newbalanceDest"]!=0) &(df["oldbalanceDest"]!=0) ].index)

if it's detected as flagged fraud so it's fraud

In [18]:
#is fraud
df=df.drop(df[(df["isFlaggedFraud"]==1) ].index)

if the amount of this oeration is 0 so it's fraud

In [19]:
#is fraud
df=df.drop(df[(df["amount"]==0) ].index)

In [20]:
df.shape

(2770377, 12)

# split and export data (static step)

to train our model we split randomly our data to dataframes with 10000 not fraud rows concatenated with 2/3 of the real frauds NOT RANDOM BUT RESPECTING THE TRUE ORDER for training set 
and we took randomly almost 2500 rows not fraud concatenated with the rest of frauds (1/3 of all frauds respecting the order)

In [21]:
#Frauds dataset
df_Fraud=df.loc[(df.isFraud == 1) ]
df_notFraud=df.loc[(df.isFraud == 0) ] 
df_Fraud.shape

(8181, 12)

In [22]:
#spliting fraud dataset to training and testing sets
df_fraud_train=df_Fraud.iloc[:int(2*len(df_Fraud)/3),:]  #1/3 for testing and 2/3 for training
df_fraud_test=df_Fraud.iloc[int(2*len(df_Fraud)/3):,:]

In [25]:


concatenated=pd.concat([df_notFraud,df_fraud_test],ignore_index=False)
concatenated.to_csv(r'drive/My Drive/statistics/parts/test_fraud_global.csv', index = True)


then we export this dataframes to our folder

In [ ]:
#split randomly our df_notFraud dataset

notFraud_copy=df_notFraud
i=0
while True:
  if notFraud_copy.shape[0]>10000:
    part=notFraud_copy.sample(frac =10000/notFraud_copy.shape[0])
    concatenated=pd.concat([part,df_fraud_train],ignore_index=False)
    concatenated.to_csv(r'drive/My Drive/statistics/parts/isFraud'+str(i)+'.csv', index = True)
    notFraud_copy=notFraud_copy.drop(part.index)
    i=i+1
    print(str(notFraud_copy.shape) + str(i) +"    "+ str(concatenated.shape))
  else:
    concatenated=pd.concat([notFraud_copy,df_fraud_test],ignore_index=False)
    concatenated.to_csv(r'drive/My Drive/statistics/parts/isFraudtest.csv', index = True)
    i=i+1
    print(str(notFraud_copy.shape) + str(i) +"    "+ str(concatenated.shape))
    print("3IIIW")
    break


# import data (static step)

we import all our dataframes and we put then in a list and we import also our testing set 

In [ ]:
dt=list()
for i in range (0,276):
  dt.append(pd.read_csv('drive/My Drive/statistics/parts/isFraud'+str(i)+'.csv'))
  dt[i]=dt[i].set_index('Unnamed: 0')
testing_set=pd.read_csv('drive/My Drive/statistics/parts/isFraudtest.csv')
testing_set=testing_set.set_index('Unnamed: 0')



# prepare data for models and fit RF

we split our training and testing set to two parts for each one
X part regroups all the independent variables and y part the dependent variables
then we will train our model using all our dataframes (Training) and we will predict using each model one prediction using our testing set 
and the final prediction will be positive if just one of the partical prediction is positive

In [ ]:
dt_X=list()
dt_y=list()
for i in range (0,276):
  dt_X.append(dt[i].iloc[:,[0,1,2,4,5,7,8,10,11]])
  dt_X[i]=pd.get_dummies(dt_X[i], prefix='t')
  dt_y.append(dt[i].iloc[:,[9]])
def test_def(set):
  XX=set.iloc[:,[0,1,2,4,5,7,8,10,11]]
  XX=pd.get_dummies(XX)
  yy=set.iloc[:,[9]]
  return XX,yy
testing_set_X,testing_set_y=test_def(testing_set)# parse your testing set here


done


our model automatically is trained using our generated datarames and is tested using our testing dataframe
but we can use those models to predect fraud possibility on new dataframes

In [ ]:
from sklearn.ensemble import RandomForestClassifier
y_pred=list()
classif_list=list()
for i in range (0,276):

  classifier = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=10,
                        n_jobs=-1, oob_score=False, random_state=None,
                        verbose=0, warm_start=False)
  classifier.fit(dt_X[i], dt_y[i].values.ravel())
  classif_list.append(classifier)
  # Predicting the Test set results
  y_pred.append(classifier.predict(testing_set_X))
  print(i)
  # Accuracy on the Test set results                              


In [ ]:
j=y_pred[0]
for i in range (1,276):
  j=j+y_pred[i]
for a in range(0, len(j)):
  if j[a]>0:
    j[a]=1
  else:
    j[a]=0
print(j)
print("{:.0%}".format(accuracy_score(testing_set_y,j)))

[0 0 0 ... 1 1 1]
98%


# exploit our model on different datasets

predict on a new data frame example

In [ ]:
input_data=testing_set
test_X,test_y=test_def(input_data)
predict=classif_list[0].predict(test_X)
for i in range (1,276):
  predict=predict+classif_list[i].predict(test_X)
  print(i)
for a in range(0, len(predict)):
  if predict[a]>0:
    predict[a]=1
  else:
    predict[a]=0
print(predict)
print("{:.0%}".format(accuracy_score(test_y,predict)) + "  is the accuracy")

exploiting of RF results

In [ ]:
#0 ARE TRUE POSITIVES AND -1 are the false positives
pd.DataFrame(test_y.values.ravel()-predict).value_counts()

 0    4807
-1     116
dtype: int64

In [ ]:
final_test=input_data.reset_index()
final_test['predicted']=predict #add a new feature which means that it's predicted positive by random forest


In [ ]:
pd.crosstab(final_test.isFraud,final_test.predicted)
#radom forest results crosstab 

predicted,0,1
isFraud,,
0,2080,116
1,0,2727


In [ ]:
#it detects all frauds included some not frauds  the important that there are no false negative
final_rf_fraud=final_test.loc[(final_test.predicted==1 )]
print(" after random forest we know that all frauds are included in this data set with this shape  "+ str(final_rf_fraud.shape) )
print("and that the true frauds are "+str(input_data.loc[(input_data.isFraud==1 )].shape[0]) + "   and the number of false positive is:  " +str(final_rf_fraud.shape[0]-input_data.loc[(input_data.isFraud==1 )].shape[0]))

 after random forest we know that all frauds are included in this data set with this shape  (2843, 14)
and that the true frauds are 2727   and the number of false positive is:  116


# fraud detect boucle

the following loop will detect the majority of frauds which exist the data which we take from random forest


In [ ]:
df_boucle=final_rf_fraud
df_repeat=df_boucle.loc[(df_boucle.isFraud == 10)]

ii=0
jj=0
for step in df_boucle.step.unique():
  print(step)
  for i in range(0,len(df_boucle[df_boucle["step"]==step])):
    for j in range(0,len(df_boucle[df_boucle["step"]==step])):
      iii=ii+i
      jjj=jj+j
      if df_boucle.amount[df_boucle.index[iii]]==df_boucle.amount[df_boucle.index[jjj]] and iii!=jjj and df_boucle.type[df_boucle.index[iii]] != df_boucle.type[df_boucle.index[jjj]]:
        df_repeat=df_repeat.append(df_boucle[df_boucle.index==df_boucle.index[iii]])

        break
  ii=ii+len(df_boucle[df_boucle["step"]==step])
  jj=jj+len(df_boucle[df_boucle["step"]==step])
print("the nshape of the frauds which are detected by this loop is : (they are all true positive)   "+str(df_repeat.shape))

In [ ]:
pd.crosstab(df_repeat.isFraud,df_repeat.predicted)
#all frauds that we predict by random forest are also predicted by this algorithm ( we use random forest to reduce the number o iteration , smaller dataset in the input of loop algorithm)

predicted,1
isFraud,
1,2722


In [ ]:
print(str(df_repeat.isFraud.value_counts()[1]) + "  is the number of frauds detected by loop algorithm and the number that it doesn't detect is:   " + str(input_data.loc[input_data.isFraud==1].shape[0]) + "  the diff is " +str(input_data.loc[input_data.isFraud==1].shape[0]-df_repeat.isFraud.value_counts()[1]) )
#our loop algorith detects only frauds but not all of them

2722  is the number of frauds detected by loop algorithm and the number that it doesn't detect is:   2727  the diff is 5


# boucles

In [ ]:
#number of true frauds
print("number of true frauds is  ")
print(df_boucle.loc[df_boucle.isFraud==1].shape[0])
print("number of detected frauds by random forest is  ")
print(df_boucle.loc[df_boucle.predicted==1].shape[0])
print("loop method detects  ")
print(str(df_repeat.loc[df_repeat.isFraud==1].shape[0]) + " as true frauds and this is the size of loop results  :  " + str(df_repeat.shape[0] )+ "   which mean that all the detected frauds by this loop are true")

number of true frauds is  
2727
number of detected frauds by random forest is  
2843
loop method detects  
2722 as true frauds and this is the size of loop results  :  2722   which mean that all the detected frauds by this loop are true


In [ ]:
expert=df_boucle.loc[df_boucle.predicted==1]
expert=expert.drop(df_repeat.index)
print("the expert should detect almost :"+str(expert.loc[expert.isFraud==1].shape[0]) + "   from the final data set of (rows will be given to this expert) " +str(expert.shape[0]))


the expert should detect almost :5   from the final data set of (rows will be given to this expert) 121
